# G27
Dave Brunner

In [23]:
import glob
import os
import torch
import string

import numpy as np
import pandas as pd
from torchinfo import summary 

from matplotlib import pyplot as plt
import matplotlib.ticker as ticker

from sklearn.metrics import confusion_matrix

### Load Data

The names can be found in text files in a src directory, one file per language.

In the following you can find some utilities to load the data into pandas data frames. 

We will restrict to some common European languages. 

With the given selection, we will identify all the occurring characters and initialize an alphabet.<br>
For this alphabet, we will use a one-hot-encoding to map them into a vector space representation. 

Foresee a suitable character for the end of the word, e.g. 'END'.

In [24]:
srcdir = 'data/names'
languages = ["English", "French", "Italian", "German", "Spanish"]

In [25]:
# inspect the data directory
def findFiles(path):
    return glob.glob(path)

In [26]:
print('\n'.join(findFiles(os.path.join(srcdir, '*.txt'))))

data/names/Czech.txt
data/names/German.txt
data/names/Arabic.txt
data/names/Japanese.txt
data/names/Chinese.txt
data/names/Vietnamese.txt
data/names/Russian.txt
data/names/French.txt
data/names/Irish.txt
data/names/English.txt
data/names/Spanish.txt
data/names/Greek.txt
data/names/Italian.txt
data/names/Portuguese.txt
data/names/Scottish.txt
data/names/Dutch.txt
data/names/Korean.txt
data/names/Polish.txt


In [27]:
# Read a file and split into lines
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return lines


def load_data(srcdir, categories=None):
    names_list = []
    for filename in findFiles(os.path.join(srcdir, '*.txt')):
        category = os.path.splitext(os.path.basename(filename))[0]
        if not categories or category in categories:
            names = readLines(filename)
            names_list.extend([(name, category) for name in names])
    df = pd.DataFrame(names_list)
    df.columns = ["name", "lang"]
    return df

In [28]:
names = load_data(srcdir, categories=languages)
names.head()

,name,lang
0,Abbing,German
1,Abel,German
2,Abeln,German
3,Abt,German
4,Achilles,German


In [29]:
maxlen = np.max([len(name) for name in names.name])
print("Maximum name length: ", maxlen)

Maximum name length:  18


In [30]:
print(f'There are {len(names)} names in the dataset')

There are 5676 names in the dataset


In [31]:
alphabet = sorted(list(set(''.join([name for name in names.name]))))
alphabet.append('END')
len_alphabet = len(alphabet)
char_index = dict((c, i) for i, c in enumerate(alphabet))
print("Size of alphabet: ", len_alphabet)
print(alphabet)

Size of alphabet:  74
[' ', "'", 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'Á', 'É', 'ß', 'à', 'á', 'ä', 'ç', 'è', 'é', 'ê', 'ì', 'í', 'ñ', 'ò', 'ó', 'ö', 'ù', 'ú', 'ü', 'END']


In [32]:
names.groupby('lang')['name'].count() / len(names)

lang
English    0.646230
French     0.048802
German     0.127555
Italian    0.124912
Spanish    0.052502
Name: name, dtype: float64

### Vector Representations

Now construct the vector representation by using one-hot-vectors. 

In [33]:
language_to_index = {country: index for index, country in enumerate(names.lang.unique())}
index_to_language = {index: country for index, country in enumerate(names.lang.unique())}


def onehot(i, length):
    v = np.zeros(length);
    v[i] = 1
    return v


def name_representation(name, maxlen):
    name_trunc = str(name)[0:maxlen]
    size = len(char_index)
    vector = [onehot(char_index[j], size) for j in str(name)]
    # fill the rest with 
    for k in range(0, maxlen - len(str(name))):
        vector.append(onehot(char_index['END'], size))
    return vector


def lang_representation(language, language_to_index):
    y = np.zeros(len(language_to_index))
    y[language_to_index[language]] = 1
    return y


def lang_from_output(score):
    return index_to_language[np.argmax(score)]


def predict(name, model):
    score = model.predict(np.array([name_representation(name, maxlen)]))[0]
    return lang_from_output(score)

### Prepare train/test

Split the data into train/test

Shuffle the data

Transform the names data into a suitable vector respresentation:
* names into numpy arrays of shape (*,maxlen,len_alphabet)
* language into numpy array of shape (*,len(languages))



In [34]:
test_split = 0.2

# Shuffle and split names data
# names = names.sample(frac=1).reset_index(drop=True)
# print(names.head())

train = names[int(len(names) * test_split):]
test = names[:int(len(names) * test_split)]

In [35]:
# Map train and test data into vector space (one-hot-vectors)
X_train = np.array([name_representation(name, maxlen) for name in train.name])
Y_train = np.array([lang_representation(lang, language_to_index) for lang in train.lang])

X_test = np.array([name_representation(name, maxlen) for name in test.name])
Y_test = np.array([lang_representation(lang, language_to_index) for lang in test.lang])

In [36]:
print(X_train.shape)
print(X_test.shape)
print(X_train[0].shape)

(4541, 18, 74)
(1135, 18, 74)
(18, 74)


Possibly, pack the data into a Dataset (e.g. when working with in PyTorch)

In [37]:
# convert from numpy to torch tensors
X_train = torch.tensor(X_train, dtype=torch.float32, requires_grad=True)
Y_train = torch.tensor(Y_train, dtype=torch.float32, requires_grad=True)
train_dataset = torch.utils.data.TensorDataset(X_train, Y_train)

X_test = torch.tensor(X_test, dtype=torch.float32, requires_grad=True)
Y_test = torch.tensor(Y_test, dtype=torch.float32, requires_grad=True)
test_dataset = torch.utils.data.TensorDataset(X_test, Y_test)

In [38]:
print(X_train[0])
lang_from_output(X_train[0].detach().numpy())
print(Y_train[0])

tensor([[0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 1.],
        [0., 0., 0.,  ..., 0., 0., 1.],
        [0., 0., 0.,  ..., 0., 0., 1.]], grad_fn=<SelectBackward0>)
tensor([0., 0., 1., 0., 0.], grad_fn=<SelectBackward0>)


In [39]:
batch_size = 128

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

### Define and Train Model: Single Layer with SimpleRNN

Create an RNN consisting of a single layer with a SimpleRNN and a softmax.

Then train the model. Play with different number of hidden units in the layer to obtain a good accuracy.

In [40]:
from torch import nn

input_size = len_alphabet
hidden_size = 32
output_size = len(languages)


class ElmanRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1, batch_first=True):
        super(ElmanRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers,
                          batch_first=batch_first)
        self.fc = nn.Linear(hidden_size, output_size)
        self.activation = nn.ReLU()

    def forward(self, x):
        out, hn = self.rnn(x)
        return self.activation(self.fc(out[:, -1, :]))


model = ElmanRNN(input_size, hidden_size, output_size)
summary(model, input_size=(batch_size, maxlen, len_alphabet))

Layer (type:depth-idx)                   Output Shape              Param #
ElmanRNN                                 [128, 5]                  --
├─RNN: 1-1                               [128, 18, 32]             3,456
├─Linear: 1-2                            [128, 5]                  165
├─ReLU: 1-3                              [128, 5]                  --
Total params: 3,621
Trainable params: 3,621
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 7.98
Input size (MB): 0.68
Forward/backward pass size (MB): 0.59
Params size (MB): 0.01
Estimated Total Size (MB): 1.29

In [41]:
batch_size = 128
n_epochs = 10
learning_rate = 0.001
model = ElmanRNN(input_size, hidden_size, output_size)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

model.train()

for epoch in range(n_epochs):
    correct = 0
    total = 0
    for i, (names, labels) in enumerate(train_loader):
        
        optimizer.zero_grad()

        outputs = model(names)
        loss = criterion(outputs, labels)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        # Get predictions from the maximum value
        _, predicted = torch.max(outputs.data, 1)
        labels = torch.max(labels, 1)[1]

        # Total number of labels
        total += labels.size(0)

        # Total correct predictions
        correct += (predicted == labels).sum()

    accuracy = 100 * correct / total
    print(
        f"Epoch [{epoch + 1}/{n_epochs}], "
        f"Loss: {loss.item():.4f}, "
        f"Accuracy: {accuracy :.4f}"
    )

Epoch [1/10], Loss: 0.7960, Accuracy: 77.8243
Epoch [2/10], Loss: 0.9035, Accuracy: 77.8243
Epoch [3/10], Loss: 0.7709, Accuracy: 77.8243
Epoch [4/10], Loss: 0.9004, Accuracy: 77.8243
Epoch [5/10], Loss: 0.9441, Accuracy: 77.8243
Epoch [6/10], Loss: 0.6414, Accuracy: 77.8243
Epoch [7/10], Loss: 0.7276, Accuracy: 77.8243
Epoch [8/10], Loss: 0.9863, Accuracy: 77.8243
Epoch [9/10], Loss: 0.9009, Accuracy: 77.8243
Epoch [10/10], Loss: 0.8146, Accuracy: 77.8243


#### Findings

...

### Implement Model with several SimpleRNN Layers

In [42]:
### START YOUR CODE

model = ...

### END YOUR CODE

In [43]:
### START YOUR CODE

# train...

### END YOUR CODE

#### Findings

...


### Class Imbalance Handling

Choose a method to address the class imbalance seen in the given example.
- minority resampling 
- class weights in the loss

Implement it and incorporate it in the training.
Evaluate the results and compare it with the results obtained with the unbalanced training.  

In [44]:
### START YOUR CODE

# train...

### END YOUR CODE